In [3]:
import numpy as np
from utils import int_to_mac

data = np.load('./data/ID_104_10s_BSG_ECG_II_Labels.npy')
print(data.shape)

data_row = data[10]
# bsg = data_row[:1000]
# ecg = data_row[1000:3560]
# ecg = resample(ecg, 1000)
mac_bsg = data_row[-54]
mac_ecg = mac_bsg + 100
mac_bsg = int_to_mac(mac_bsg)
mac_ecg = int_to_mac(mac_ecg)
print(mac_bsg, mac_ecg)

(16399, 3615)
30:30:f9:73:4c:34 30:30:f9:73:4c:98
